In [1617]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import selenium
from tqdm import tqdm
import time

In [2]:
from selenium import webdriver   # driver de selenium
from selenium.webdriver.chrome.options import Options   # opciones del driver
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.common.keys import Keys  # para usar teclado
from selenium.webdriver.chrome.service import Service

Starting scrapping process getting all companies and scores available

In [1379]:
serv = Service('/Users/paloma/Desktop/Ironhack/proyecto_final/chromedriver')

In [1380]:
opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('user-data-dir=cookies')

In [488]:
#opening google chrome
driver = webdriver.Chrome(service = serv, options = opciones)

In [489]:
#accessing website
driver.get('https://www.sustainalytics.com/esg-ratings')

In [180]:
def next_page(range_start,range_end,jump):
    paginas = [str(x) for x in list(range(range_start,range_end,jump))]
    for x in paginas:
        driver.find_element(By.LINK_TEXT, x).click()
        time.sleep(1)

In [199]:
def scrap_current_page():
    table = driver.find_element(By.ID, 'company_ratings')
    html = table.get_attribute('outerHTML')
    soup = bs(html, 'html.parser')
    #extracting lists of the different elements
    companies.extend([result.get_text() for result in (soup.find_all('a')[:10])])
    symbols.extend([result.get_text() for result in (soup.find_all('small'))])
    ratings.extend([result.get_text() for result in (soup.find_all('div', class_='col-2'))])
    risk.extend([result.get_text() for result in (soup.find_all('div', class_="col-lg-6 col-md-10"))])

In [145]:
#first we extract the first page:
#selecting table and creating soup
table = driver.find_element(By.ID, 'company_ratings')
html = table.get_attribute('outerHTML')
soup = bs(html, 'html.parser')

In [146]:
#extracting lists of the different elements
companies = []
symbols = []
ratings = []
risk = []

companies.extend([result.get_text() for result in (soup.find_all('a')[:10])])
symbols.extend([result.get_text() for result in (soup.find_all('small'))])
ratings.extend([result.get_text() for result in (soup.find_all('div', class_='col-2'))])
risk.extend([result.get_text() for result in (soup.find_all('div', class_="col-lg-6 col-md-10"))])

In [147]:
len(companies)

10

In [583]:
companies = df['companies'].to_list()
symbols = df['symbols'].to_list()
ratings = df['ratings'].to_list()
risk = df['risk'].to_list()

In [640]:
#extracting the rest of the pages
paginas = [str(x) for x in list(range(2,1381))]
len(paginas)

1379

In [592]:
%%time
for x in paginas:
    driver.find_element(By.LINK_TEXT, x).click()       
    #selecting table and creating soup
    table = driver.find_element(By.ID, 'company_ratings')
    html = table.get_attribute('outerHTML')
    soup = bs(html, 'html.parser')
    #extracting lists of the different elements
    companies.extend([result.get_text() for result in (soup.find_all('a')[:10])])
    symbols.extend([result.get_text() for result in (soup.find_all('small'))])
    ratings.extend([result.get_text() for result in (soup.find_all('div', class_='col-2'))])
    risk.extend([result.get_text() for result in (soup.find_all('div', class_="col-lg-6 col-md-10"))])
    time.sleep(3) #allowing some delay so that the site can charge the page and the driver can find the next page button

CPU times: user 2.35 s, sys: 199 ms, total: 2.54 s
Wall time: 25min 4s


In [555]:
len(companies),len(symbols),len(ratings),len(risk)

(14371, 14371, 14371, 14371)

In [579]:
next_page(700,245,-2)

In [552]:
scrap_current_page()

In [603]:
#since we get some errors and lose some information, we are extracting the information twice, merging and eliminating duplicates
df= pd.DataFrame()
df['Companies'] = companies
df['Symbols'] = symbols
df['Ratings'] = ratings
df['Risk'] = risk
print(len(df[df.duplicated(subset='companies') == True]))
df.drop_duplicates(keep = 'first', inplace = True)


ValueError: Cannot set a DataFrame with multiple columns to the single column Companies

In [553]:
esg = pd.DataFrame()

In [554]:
esg['Companies'] = companies
esg['Symbols'] = symbols
esg['Ratings'] = ratings
esg['Risk'] = risk
print(len(esg[esg.duplicated(subset='Companies') == True]))
esg.drop_duplicates(subset='Companies', inplace = True)

In [636]:
companies_esg = pd.concat([esg,df])

In [641]:
companies_esg = companies_esg.drop_duplicates(subset =  'Companies')
len(companies_esg)

13791

In [643]:
#companies_esg.to_csv('companies_esg.csv', index = False)

Scrapping rest of data using company Identifiers:

In [846]:
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [1205]:
#Separating companies with and without Identifier
names = (companies_esg['Companies']).to_list()

In [1326]:
from random import shuffle

In [1328]:
names_r = names[873:]
shuffle(names_r)
len(names_r)

In [1546]:
replace_symb ={' Co.':'', ' Ltd.':'',' Inc.':'', ' Plc':'', ' Corp.':'', ' CO. LTD.':'', ' Corp.':''}
names_r = [w.replace(' Corp.','') for w in names_r]

In [1615]:
driver.quit()

In [1578]:
opciones.page_load_strategy = 'none'
opciones.add_argument('--disk-cache-dir=/path/to/cache')
wait = WebDriverWait(driver, 20)
driver = webdriver.Chrome(service = serv, options = opciones)

#creating empty lists to save scrapes and not lose them if scrapping process fails:
industry = []
country = []
symbol = []
company_name = []

In [1579]:
#Scrapping industry and country from the same website, we start the scrapping indicating the first website to start:
driver.get('https://www.sustainalytics.com/esg-rating/shinko-shoji-co-ltd/1008754046')

In [1659]:
#AHORA ESTO NO!!!
#scrapping current website:
table = driver.find_element(By.CLASS_NAME, 'col-md-8.col-sm-12.company-data')
html = table.get_attribute('outerHTML')
soup = bs(html, 'html.parser')
company_name.append((soup.find('h2')).get_text())
industry.append((soup.find('strong', class_='industry-group')).get_text())
country.append((soup.find('strong', class_='country')).get_text())
symbol.append((soup.find('strong', class_='identifier')).get_text())

In [1614]:
for s in names_r[895:1000]:     #remember changing first row index for the NEXT website to be scrapped, not the current one
    
    #changing url to next company
    #if everything goes well:
    try: 
        textarea = driver.find_element(By.ID, 'searchInput')
        textarea.send_keys(s)
        time.sleep(3)
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'search-link.js-fix-path')))
        element.click()
    #if website takes too long to charge or there is any error:
    except:
        driver.refresh()
        time.sleep(10)
        textarea = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'searchInput')))
        driver.execute_script("window.stop();")
        textarea.send_keys(s)
        time.sleep(3)
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'search-link.js-fix-path')))
        element.click()
    #stop charging website once we have the information we need:
    time.sleep(10)
    element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'searchInput')))
    driver.execute_script("window.stop();")

    #scrapping elemnts and adding them to lists
    
    table = driver.find_element(By.CLASS_NAME, 'col-md-8.col-sm-12.company-data')
    html = table.get_attribute('outerHTML')
    soup = bs(html, 'html.parser')
    company_name.append((soup.find('h2')).get_text())
    industry.append((soup.find('strong', class_='industry-group')).get_text())
    country.append((soup.find('strong', class_='country')).get_text())
    symbol.append((soup.find('strong', class_='identifier')).get_text())

    

KeyboardInterrupt: 

In [1648]:
len(company_name)

1208

In [1650]:
#last company scrapped:
companies_esg[companies_esg['Companies'] == (company_name[-1])]

,Companies,Symbols,Ratings,Risk
5398,HDFC Bank Ltd.,BOM:500180,30.8,High ESG Risk


In [1611]:
#next company:
names_r.index("Cameco")

894

In [1662]:
backup = pd.DataFrame()
backup['Industry']=industry
backup['Country']=country
backup['Symbols']=symbol
backup['Companies']=company_name
#backup.to_csv('backup_3.csv', index = False)

In [1627]:
#recuperar backup
backup = pd.read_csv('data/working data/backup_3.csv')
industry = backup['Industry'].to_list()
country = backup['Country'].to_list()
symbol = backup['Symbols'].to_list()
company_name = backup['Companies'].to_list()

In [1626]:
#sacando info de empresas de las que tenemos detalles ya, para agilizar

In [1625]:
us_add_comp = pd.read_excel('data/working data/esg_score_all.xlsx')
us_add = (us_add_comp['Company Name'][115:]).to_list()

In [1628]:
driver.quit()
serv = Service('/Users/paloma/Desktop/Ironhack/proyecto_final/chromedriver')
driver = webdriver.Chrome(service = serv, options = opciones)
driver.get('https://www.sustainalytics.com/esg-rating/shinko-shoji-co-ltd/1008754046')

In [1661]:
for s in us_add[104:]:     #remember changing first row index for the NEXT website to be scrapped, not the current one
    
    #changing url to next company
    #if everything goes well:
    try: 
        textarea = driver.find_element(By.ID, 'searchInput')
        textarea.send_keys(s)
        time.sleep(3)
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'search-link.js-fix-path')))
        element.click()
    #if website takes too long to charge or there is any error:
    except:
        driver.refresh()
        time.sleep(10)
        textarea = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'searchInput')))
        driver.execute_script("window.stop();")
        textarea.send_keys(s)
        time.sleep(3)
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'search-link.js-fix-path')))
        element.click()
    #stop charging website once we have the information we need:
    time.sleep(10)
    element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'searchInput')))
    driver.execute_script("window.stop();")

    #scrapping elemnts and adding them to lists
    
    table = driver.find_element(By.CLASS_NAME, 'col-md-8.col-sm-12.company-data')
    html = table.get_attribute('outerHTML')
    soup = bs(html, 'html.parser')
    company_name.append((soup.find('h2')).get_text())
    industry.append((soup.find('strong', class_='industry-group')).get_text())
    country.append((soup.find('strong', class_='country')).get_text())
    symbol.append((soup.find('strong', class_='identifier')).get_text())

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=122.0.6261.112)
Stacktrace:
0   chromedriver                        0x00000001030852f8 chromedriver + 4625144
1   chromedriver                        0x000000010307cea3 chromedriver + 4591267
2   chromedriver                        0x0000000102c7bc97 chromedriver + 392343
3   chromedriver                        0x0000000102cc8578 chromedriver + 705912
4   chromedriver                        0x0000000102cbc427 chromedriver + 656423
5   chromedriver                        0x0000000102ce9872 chromedriver + 841842
6   chromedriver                        0x0000000102cbbdb8 chromedriver + 654776
7   chromedriver                        0x0000000102ce9a2e chromedriver + 842286
8   chromedriver                        0x0000000102d0868f chromedriver + 968335
9   chromedriver                        0x0000000102ce9613 chromedriver + 841235
10  chromedriver                        0x0000000102cba3da chromedriv